# Create Agent via Boto3 SDK

## Introduction

This notebook demonstrates the usage of the `bedrock-agent` and `bedrock-agent-runtime` boto3 clients to:
- Create an agent
- Create an action group
- Associate the agent with the action group and prepare it for use
- Create an agent alias
- Invoke the agent

We'll be utilizing Bedrock's Claude v2.1 through the Boto3 API.

**Note:** *This notebook is designed to be run both within and outside of an AWS environment.*

### Prerequisites

Ensure you have an AWS account with permissions to:
- Create and manage IAM roles and policies
- Create and invoke AWS Lambda functions
- Create, read from, and write to Amazon S3 buckets
- Access and manage Amazon Bedrock agents and models


### Context

The following sections guide you through creating and invoking a Bedrock agent using the Boto3 SDK.

### Use Case

The AI Fashion Assistant is an intelligent agent that helps users find the perfect apparel and outfits based on their preferences and context. By analyzing user-provided query images, the agent searches its catalog to find matching or similar apparel items. When location information is available, the agent factors in weather conditions to suggest appropriate outfits. The AI Fashion Assistant can also modify images, either by updating the outfits while preserving the background scene through inpainting, or by changing the background while keeping the original outfits intact via outpainting. This innovative agent streamlines the process of discovering and visualizing fashion styles tailored to individual needs.


![flowchart_agent](images/flowchart_agent.png)


The Agent is designed to:
- Receive and validate user query images
- Search for and retrieve matching apparel images from a catalog
- Provide suggestions for similar styles if a match is not found
- Suggest proper outfits based on weather if location information is provided
- Inpaint the image to update the outfits without changing the scene
- Outpaint the image to update the background without changing the outfits


##### Pre-requisite

In [ ]:
upgrade_output = !pip install --upgrade pip
install_boto3_output = !pip install boto3
##Ensure your boto3 and botocore libraries are up to date
upgrade_output_botocore_boto3= !pip install --upgrade boto3 botocore 
!pip install streamlit --quiet

##### Import config setup

In [ ]:
from dependencies.config import *

##### Create Required Infrastructure

In [ ]:
!python ./dependencies/build_infrastructure.py

##### Create Agent

In [ ]:
from dependencies.config import *

list_agent=bedrock_agent_client.list_agents()['agentSummaries']
#print(list_agent)
#print(agent_name)
# Search f
agent_id = next((agent['agentId'] for agent in list_agent if agent['agentName'] == agent_name), None)

print(agent_id)

response = bedrock_agent_client.list_agent_aliases(
    agentId=agent_id,
)
response['agentAliasSummaries']
agentAliasId=next((agent['agentAliasId'] for agent in response['agentAliasSummaries'] if agent['agentAliasName'] == agent_alias_name), None)
agent_alias_id=agentAliasId
print(agent_alias_id)

###### Store Variables to Local File

In [ ]:
import pickle

variables = {
    'agent_id': agent_id,
    'agent_alias_id': agent_alias_id,
    'bucket_name': bucket_name
}

# Store variables to local
with open('variables.pkl', 'wb') as f:
    pickle.dump(variables, f)

##### Create Dependency File for Streamlit UI

In [ ]:
import os
from io import open

# Create .streamlit folder
if not os.path.exists('.streamlit'):
    os.makedirs('.streamlit')


# Create config.toml file
config_file_path = os.path.join('.streamlit', 'config.toml')
with open(config_file_path, 'w', encoding='utf-8') as f:
    f.write('[server]\nenableXsrfProtection = false\nenableCORS = false\n')

print("Configuration file created successfully.")

#### UI Interaction

The user can then connect to streamlit UI and interact with the created agent

## Clean up (optional)

The next steps are optional and delete the infrastructure we built. 



In [ ]:
!python ./dependencies/clean.py


## Conclusion
We have now experimented with using `boto3` SDK to create, invoke and delete an agent.

### Take aways
- Adapt this notebook to interact with your fashion agent!

## Thank You